In [53]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
rootPath = "../../../../"#change your rootpath here.
dataset=pd.read_csv(rootPath+"Datasets/insurance_pre.csv")

In [55]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [ ]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.astype(int)

In [59]:
indep=dataset[["age","bmi","children","sex_male","smoker_yes"]]
dep=dataset["charges"]

In [60]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid={'criterion':['squared_error','friedman_mse','absolute_error','poisson'],'max_features':['sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1,error_score='raise')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(error_score='raise', estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             verbose=3)

In [63]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

In [64]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_Score value for best parameter{}:".format(grid.best_params_),r_score)

The R_Score value for best parameter{'criterion': 'absolute_error', 'max_features': 'log2', 'n_estimators': 100}: 0.8683660877825695


In [ ]:
table=pd.DataFrame.from_dict(re)
table

In [66]:
import pickle
filename=rootPath+"Deployed/RFGrid.sav"

In [67]:
pickle.dump(grid,open(filename,'wb'))
pickle.dump(sc,open(rootPath+"Deployed/StandardScaler.sav",'wb'))

In [68]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

In [80]:
loaded_model=pickle.load(open(filename,'rb'))
sc=pickle.load(open(rootPath+"Deployed/StandardScaler.sav",'rb'))
data=[[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]]
df=pd.DataFrame(data,columns=['age','bmi','children','sex_male','smoker_yes'])
preinput=sc.transform(df)
Future_Prediction=loaded_model.predict(preinput)
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[7497.63]
